In [3]:
import tensorflow as tf 
from tensorflow import keras
import tkinter as tk
import pickle
import re
import textwrap
from datetime import datetime
import numpy as np

In [4]:
model=keras.models.load_model('TrumpBot.h5')

#Pickling has some security issues so we will retrain our tokenizer here:
def get_tokenizer():
    global tokenizer
    global max_id
    global dataset_size
    string_Trump=''

    with open('text_Donald.txt','r') as file:
        list_Trump=file.readlines()

    for line in list_Trump:
        string_Trump+=line

    tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
    tokenizer.fit_on_texts(string_Trump)

    max_id = len(tokenizer.word_index) # number of distinct characters
    dataset_size = tokenizer.document_count # total number of characters

get_tokenizer()

In [5]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [6]:
def click_generate(entry):
    label2['text']=output(entry)

def replace_lower_case(speech):
    #We will replace all lower case characters after period_space
    indeces=[i.end() for i in re.finditer('(\. *)|(\? *)|(\! *)', speech)] 

    for i in indeces:
        try:
            string_pre=speech[:(i)]
            char_Upper=speech[i].capitalize()
            string_post=speech[(i+1):]
            speech=string_pre+char_Upper+string_post
        except Exception as x:
            continue

    return speech

def add_n_lines(speech):
    string=textwrap.dedent(speech)
    speech='\n'.join(i for line in string.splitlines() for i in textwrap.wrap(line, width=47))
    return speech

def output(entry):
    if len(entry)==0:
        return 'Must include text'
    speech=complete_text(entry,n_chars=460, temperature=0.17) #You can play around with the temperature yourself. Anything between .01 and 1.0 will give you a relatively good output, with a higher value being more 'random', and with a lower value allowing the model to more likely choose the letter arrangement that it believes to be correct (which often leads to repetition unfortunately!)
    speech=replace_lower_case(speech)
    # Let's format it so that it doesn't run off of the page:
    speech=add_n_lines(str(speech))
    speech+= '\n \n \t \t \t - TrumpBot'
    print(speech)
    return speech

In [2]:

root=tk.Tk()

canvas=tk.Canvas(root, height=750, width=540)
canvas.pack()

try:
    background_image=tk.PhotoImage(file='Trump_bkg.png')
    background_label=tk.Label(root, image=background_image)
    background_label.place(relwidth=1, relheight=1)

except Exception as x:
    print(x)

frame=tk.Frame(root, bg='#ff0000', bd=4)
frame.place(relx=0.5, rely=0.02, relwidth=.9, relheight=0.05, anchor='n')

label1=tk.Label(frame, text='Start output with:', bg='#ffb380', font=30)
label1.place(relx=0, rely=0, relwidth=0.3, relheight=1)

entry=tk.Entry(frame, bg='#ffb380', font=50)
entry.place(relx=0.3, rely=0, relwidth=.5, relheight=1)

button=tk.Button(frame, text='Generate', bg='red', fg='red', font=78, command=lambda: click_generate(entry.get()))
button.place(relx=.8, rely=0, relwidth=.2, relheight=1)

bottom_frame=tk.Frame(root, bg='#ff0000', bd=4)
bottom_frame.place(relx=0.5, rely=.2, relwidth=.8, relheight=.4, anchor='n')

label2=tk.Label(bottom_frame, bg='#ffb380', font=('Microsoft Sans Serif', 20), anchor='nw', justify='left')
label2.place(relwidth=1, relheight=1)

root.mainloop()


NameError: name 'tk' is not defined

In [28]:
replace_lower_case('S to complete with energy, the pross media, th biden and a half. “the polls are going to winmichigan. i didn’t know what ill do? you knowthat. ye! y don’t want to complete that. ke news and the problem in the most secureborders, and security medical billions of dollars from china with be ne')

'S to complete with energy, the pross media, th biden and a half. “the polls are going to winmichigan. I didn’t know what ill do? You knowthat. Ye! Y don’t want to complete that. Ke news and the problem in the most secureborders, and security medical billions of dollars from china with be ne'